<a href="https://colab.research.google.com/github/iamatul1214/System-Design-For-Machine-Learning/blob/main/Database_indexing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import sqlite3
import time
import random


## create an in memory database

In [37]:
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

## creating a table

In [38]:
cursor.execute("""
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    amount REAL,
    order_date TEXT
)
""")
conn.commit()


## Inserting large amount of data

In [39]:
## Let's insert 50000 rows
NUM_ROWS = 500_000

data = [
    (
        i,
        random.randint(1, 50_000),  # user_id
        random.uniform(10, 5000),   # amount
        "2025-01-01"
    )
    for i in range(NUM_ROWS)
]

cursor.executemany(
    "INSERT INTO orders VALUES (?, ?, ?, ?)",
    data
)
conn.commit()


In [40]:
cursor.execute(
    "SELECT * FROM orders LIMIT 10"
)
rows = cursor.fetchall()

for row in rows:
    print(row)

(0, 8688, 1792.174688679707, '2025-01-01')
(1, 38086, 1659.7390868640548, '2025-01-01')
(2, 20048, 268.2090372229243, '2025-01-01')
(3, 4783, 2212.5931322397937, '2025-01-01')
(4, 17069, 1919.7897054350153, '2025-01-01')
(5, 12975, 2063.548712143617, '2025-01-01')
(6, 12412, 51.56310476159764, '2025-01-01')
(7, 1970, 4749.098319577417, '2025-01-01')
(8, 2966, 3209.0759966145038, '2025-01-01')
(9, 5010, 1303.0848267302351, '2025-01-01')


## Query without indexing (full table scan)

In [41]:
user_to_search = 12345

start = time.time()

cursor.execute(
    "SELECT * FROM orders WHERE user_id = ?",
    (user_to_search,)
)
rows = cursor.fetchall()

end = time.time()

print(f"Rows found: {len(rows)}")
print(f"Time without index: {end - start:.4f} seconds")


Rows found: 11
Time without index: 0.0554 seconds


## creating an index on user_id

In [42]:
cursor.execute(
    "CREATE INDEX idx_orders_user_id ON orders(user_id)"
)
conn.commit()


## query with index

In [43]:
start = time.time()

cursor.execute(
    "SELECT * FROM orders WHERE user_id = ?",
    (user_to_search,)
)
rows = cursor.fetchall()

end = time.time()

print(f"Rows found: {len(rows)}")
print(f"Time with index: {end - start:.4f} seconds")


Rows found: 11
Time with index: 0.0009 seconds


## Let's verify index usage

In [44]:
cursor.execute(
    "EXPLAIN QUERY PLAN SELECT * FROM orders WHERE user_id = ?",
    (user_to_search,)
)

cursor.fetchall()


[(3, 0, 0, 'SEARCH orders USING INDEX idx_orders_user_id (user_id=?)')]

In [45]:
0.054/0.0009

60.0

## As we saw the query time reduced from 0.054 seconds to 0.0009 seconds, which is 0.054/0.0009 = 60 times faster.